# Desplegar backend en HuggingFace


En este post vamos a ver cómo desplegar un backend en HuggingFace. Vamos a ver cómo hacerlo de dos maneras, mediante la forma común, creando una aplicación con Gradio, y mediante un opción diferente usando FastAPI y Docker

Para ambos casos va a ser necesario tener una cuenta en HuggingFace, ya que vamos a desplegar el backend en un space de HuggingFace.

## Desplegar backend con Gradio

### Crear space

Primero de todo, creamos un nuevo espacio en HuggingFace.

 * Ponemos un nombre, una descripción y elegimos la licencia.
 * Elegimos Gradio como el tipo de SDK. Al elegir Gradio, nos aparecerán plantillas, así que elegimos la plantilla de chatbot.
 * Seleccionamos el HW en el que vamos a desplegar el backend, yo voy a elegir la CPU gratuita, pero tú elige lo que mejor consideres.
 * Y por último hay que elegor si queremos crear el espacio público o privado.

![backend gradio - create space](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/backend-gradio-create-space.webp)

### Código

Al desplegar el space, podemos clonarlo o podemos ver los archivos en la propia página de HuggingFace. Podemos ver que se han creado 3 archivos, `app.py`, `requirements.txt` y `README.md`. Así que vamos a ver qué poner en cada uno

#### app.py

Aquí tenemos el código de la aplicación. Cómo hemos elegido la plantilla de chatbot, ya tenemos mucho hecho, pero vamos a tener que cambiar 2 cosas, primero el modelo de lenguaje y el system prompt

Como modelo de lenguaje veía ``HuggingFaceH4/zephyr-7b-beta``, pero vamos a usar ``HuggingFaceTB/SmolLM2-1.7B-Instruct``, que es un modelo muy capaz con solo 1.7B de parámetros.

¿Por qué usamos este modelo? Como hemos elegido un space con un HW gratuito, no vamos a tener mucha potencia de cómputo, por lo que elegimos un modelo pequeño, pero aceptabel. En tu caso, si quieres un modelo más grande y mejor tienes dos opciones, dejar el mismo HW sabiendo que la inferencia va a ser mucho más lenta, o cambiar el HW por uno más potente, pero de pago.

Así que busca el texto ``client = InferenceClient("HuggingFaceH4/zephyr-7b-beta")`` y reemplázalo por ``client = InferenceClient("HuggingFaceTB/SmolLM2-1.7B-Instruct")``, o espera que más adelante pondré todo el código.

También vamos a cambiar el system prompt, que por defecto es ``You are a friendly Chatbot.``, pero como es un modelo entrenado en su mayoría en inglés, es probable que si le hablas en otro idioma te responda en inglés, así que vamos a cambiarlo por ``You are a friendly Chatbot. Always reply in the language in which the user is writing to you.``.

Así que busca el texto ``gr.Textbox(value="You are a friendly Chatbot.", label="System message"),`` y reemplázalo por ``gr.Textbox(value="You are a friendly Chatbot. Always reply in the language in which the user is writing to you.", label="System message"),``, o espera a que ahora voy a poner todo el código.

``` python
import gradio as gr
from huggingface_hub import InferenceClient

"""
For more information on `huggingface_hub` Inference API support, please check the docs: https://huggingface.co/docs/huggingface_hub/v0.22.2/en/guides/inference
"""
client = InferenceClient("HuggingFaceTB/SmolLM2-1.7B-Instruct")


def respond(
    message,
    history: list[tuple[str, str]],
    system_message,
    max_tokens,
    temperature,
    top_p,
):
    messages = [{"role": "system", "content": system_message}]

    for val in history:
        if val[0]:
            messages.append({"role": "user", "content": val[0]})
        if val[1]:
            messages.append({"role": "assistant", "content": val[1]})

    messages.append({"role": "user", "content": message})

    response = ""

    for message in client.chat_completion(
        messages,
        max_tokens=max_tokens,
        stream=True,
        temperature=temperature,
        top_p=top_p,
    ):
        token = message.choices[0].delta.content

        response += token
        yield response


"""
For information on how to customize the ChatInterface, peruse the gradio docs: https://www.gradio.app/docs/chatinterface
"""
demo = gr.ChatInterface(
    respond,
    additional_inputs=[
        gr.Textbox(value="You are a friendly Chatbot. Always reply in the language in which the user is writing to you.", label="System message"),
        gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="Max new tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(
            minimum=0.1,
            maximum=1.0,
            value=0.95,
            step=0.05,
            label="Top-p (nucleus sampling)",
        ),
    ],
)


if __name__ == "__main__":
    demo.launch()
```

#### requirements.txt

Este es el archivo en el que estarán escritas las dependencias, pero para este caso va a ser muy sencillo:

``` txt
huggingface_hub==0.25.2
```

#### README.md

Este es el archivo en el que vamos a poner la información del espacio. En los spaces de HuggingFace, al inicio de los readmes, se pone un código para que HuggingFace sepa cómo mostrar la miniatura del espacio, qué fichero tiene que usar para ejecutar el código, version del sdk, etc.

``` md
---
title: SmolLM2
emoji: 💬
colorFrom: yellow
colorTo: purple
sdk: gradio
sdk_version: 5.0.1
app_file: app.py
pinned: false
license: apache-2.0
short_description: Gradio SmolLM2 chat
---

An example chatbot using [Gradio](https://gradio.app), [`huggingface_hub`](https://huggingface.co/docs/huggingface_hub/v0.22.2/en/index), and the [Hugging Face Inference API](https://huggingface.co/docs/api-inference/index).
```

### Despliegue

Si hemos clonado el espacio, tenemos que hacer un commit y un push. Si hemos modificado los archivos en HuggingFace, con guardarlos es suficiente.

Así que cuando estén los cambios en HuggingFace, tendremos que esperar unos segundos para que se construya el espacio y podremos usarlo.

![backend gradio - chatbot](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/backend-gradio-chatbot.webp)

### Backend

Muy bien, hemos hecho un chatbot, pero no era la intención, aquí habíamos venido a hacer un backend! Para, para, fíjate lo que pone debajo del chatbot

![backend gradio - Use via API](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/backend-gradio-chatbot-edited.webp)

Podemos ver una texto ``Use via API``, donde si pulsamos se nos abre un menú con una API para poder usar el chatbot.

![backend gradio - API](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/backend%20gradio%20-%20API.webp)

Vemos que nos da una documentación de cómo usar la API, tanto con Python, con JavaScript, como con bash.

### Prueba de la API

Usamos el código de ejemplo de Python.

In [6]:
%%time

from gradio_client import Client

client = Client("Maximofn/SmolLM2")
result = client.predict(
		message="Hola, ¿cómo estás?",
		system_message="You are a friendly Chatbot. Always reply in the language in which the user is writing to you.",
		max_tokens=512,
		temperature=0.7,
		top_p=0.95,
		api_name="/chat"
)
print(result)

Loaded as API: https://maximofn-smollm2.hf.space ✔
Hola! Estoy bien, gracias por preguntar. ¿Y usted?
CPU times: user 77.3 ms, sys: 11.6 ms, total: 88.9 ms
Wall time: 5.26 s


He añadido un `%%time` para que se vea el tiempo que tarda en responder. Hay que tener en cuenta el tiempo de inferencia y el tiempo de conexión.